In [15]:
from price_loaders.tradingview import load_asset_price
import pandas as pd
from datetime import date

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

In [24]:
currYear = 2026
lb = 5

relPrice = pd.DataFrame()
stacked_TS = go.Figure()

for i in range(lb):
    CCM = load_asset_price(f"CCMH{currYear-i}", 1000, "1D")[['time', 'close']]
    CCM['time'] = CCM['time'].dt.tz_convert("America/Sao_Paulo").apply(lambda x: date(x.year, x.month, x.day))
    
    sbPgua = pd.read_excel('DATA/soja.xlsx', sheet_name='SBPRSP', skiprows=[0,1,2], header=None)
    sbPgua.columns = ['time', 'close']
    sbPgua['time'] = sbPgua['time'].apply(lambda x: date(x.year, x.month, x.day))

    aux = CCM.merge(sbPgua, how = 'inner', on = 'time', suffixes=('_co', '_sb'))
    aux['rel_price'] = aux['close_sb']/aux['close_co']

    aux['time'] = pd.to_datetime(aux['time'])
    # Garante que não hajam datas de 29 de fevereiro
    aux = aux[~((aux['time'].dt.month == 2) & (aux['time'].dt.day == 29))]
    # Ajusta o ano sem dar erro
    aux['time'] = aux['time'].apply(lambda x: x.replace(year=x.year + i))


    if i != 0:
        stacked_TS.add_trace(go.Scatter(x=aux['time'], y=aux['rel_price'], name=f'SB Pguá/CCMH{currYear-i}',line=dict(width=3)))
    else: 
        stacked_TS.add_trace(go.Scatter(x=aux['time'], y=aux['rel_price'], name=f'SB Pguá/CCMH{currYear-i}',line=dict(width=5)))


    #relPrice = pd.concat([relPrice, aux])

stacked_TS.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0.01
        )
    )

stacked_TS.show()